In [10]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [11]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
%cd /content/drive/MyDrive
!git clone https://<github_token>@github.com/Thomas-Kragerud/EECS282-Final-Project.git
%cd EECS282-Final-Project/diffusers
#!git fsck && git gc --prune=now
#!mkdir out

/content/drive/MyDrive
Cloning into 'EECS282-Final-Project'...
remote: Enumerating objects: 3114, done.
remote: Counting objects: 100% (3114/3114), done.
remote: Compressing objects: 100% (2437/2437), done.
remote: Total 3114 (delta 624), reused 3109 (delta 619), pack-reused 0
Receiving objects: 100% (3114/3114), 15.34 MiB | 12.52 MiB/s, done.
Resolving deltas: 100% (624/624), done.
Updating files: 100% (1089/1089), done.
/content/drive/MyDrive/EECS282-Final-Project/diffusers


In [12]:
!git checkout devMorten
!git pull

remote: Enumerating objects: 18, done.
remote: Counting objects: 100% (18/18), done.
remote: Compressing objects: 100% (8/8), done.
remote: Total 13 (delta 4), reused 13 (delta 4), pack-reused 0
Unpacking objects: 100% (13/13), 5.89 MiB | 7.18 MiB/s, done.
From https://github.com/Thomas-Kragerud/EECS282-Final-Project
   b42395d..9b476d3  devMorten  -> origin/devMorten
Updating b42395d..9b476d3
Fast-forward
 diffusers/examples/dreambooth/train_dreambooth_lora.py | 132 ++++++++++++++++++++++++++++++++++++-
 diffusers/shoe_images/shoe1.jpg                        | Bin 0 -> 284219 bytes
 diffusers/shoe_images/shoe2.png                        | Bin 0 -> 2176568 bytes
 diffusers/shoe_images/shoe3.png                        | Bin 0 -> 978535 bytes
 diffusers/shoe_images/shoe4.png                        | Bin 0 -> 1187926 bytes
 diffusers/shoe_images/shoe5.png                        | Bin 0 -> 1198196 bytes
 diffusers/shoe_images/shoe6.jpeg                       | Bin 0 -> 462292 bytes
 7 file

In [5]:
!pip install -e .


Obtaining file:///content/drive/MyDrive/EECS282-Final-Project/diffusers
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for diffusers (pyproject.toml) ... done
  Created wheel for diffusers: filename=diffusers-0.24.0.dev0-0.editable-py3-none-any.whl size=11038 sha256=e837fda0e59153fd58888ccb5d3444494298166d4d0863beb1ad50fc230a435b
  Stored in directory: /tmp/pip-ephem-wheel-cache-peym_q9r/wheels/8c/7f/87/4b13388e70b4fb14c9a6fae019e3df48450381ff14cc092531
Successfully built diffusers


In [ ]:
#!git pull

Already up to date.


In [13]:
%cd examples/dreambooth
!pip install -r requirements.txt
!pip install wandb
!pip install pytorch-ignite
!pip install clean-fid

# Some temp bugfix from github


# !pip install accelerate
# !pip install torchvision
# !pip install transformers
# !pip install ftfy
# !pip install tensorboard
# !pip install Jinja2

[Errno 2] No such file or directory: 'examples/dreambooth'
/content/drive/MyDrive/EECS282-Final-Project/diffusers/examples/dreambooth


In [7]:
!accelerate config default


accelerate configuration saved at /root/.cache/huggingface/accelerate/default_config.yaml


In [15]:
import os

lora_rank = 10
os.environ['MODEL_NAME'] = "runwayml/stable-diffusion-v1-5"
os.environ['INSTANCE_DIR'] = "input_shoes"
os.environ['OUTPUT_DIR'] = "out_model_data"  # Change this to your desired output directory.
os.environ['class_data_dir'] = "class_data_dir"
os.environ['class_validation_dir'] = "class_validation_dir"
os.environ['lora_rank'] = str(lora_rank)
os.environ['validation_data_dir'] = "validation_images"
os.environ["instance_validation_dir"] = "instance_validation_dir"

!rm -rf $OUTPUT_DIR
!rm -rf $class_validation_dir
!rm -rf $validation_data_dir
!rm -rf $instance_validation_dir
!rm -rf $class_data_dir

!mkdir $OUTPUT_DIR

In [ ]:


#!huggingface-cli login

!accelerate launch train_dreambooth_lora.py \
  --pretrained_model_name_or_path=$MODEL_NAME \
  --instance_data_dir=$INSTANCE_DIR \
  --output_dir=$OUTPUT_DIR \
  --instance_prompt="a photo of sks shoe" \
  --instance_validation_dir=$instance_validation_dir \
  --resolution=512 \
  --train_batch_size=1 \
  --gradient_accumulation_steps=1 \
  --checkpointing_steps=100 \
  --learning_rate=1e-4 \
  --report_to="wandb" \
  --lr_scheduler="constant" \
  --lr_warmup_steps=0 \
  --max_train_steps=1000 \
  --validation_prompt="A photo of sks shoe in a dessert" \
  --validation_epochs=1 \
  --seed="0" \
  --with_prior_preservation \
  --prior_loss_weight=1 \
  --num_class_images=100 \
  --num_validation_images=25 \
  --class_data_dir=$class_data_dir \
  --class_prompt="a photo of shoe" \
  --class_validation_dir=$class_validation_dir \
  --validation_data_dir=$validation_data_dir \
  --rank=$lora_rank \
  --calculate_DINO=True \
  --calculate_KID=True \

summary_name = f"summary_rank_{lora_rank}.zip"
os.environ['SUMMARY_NAME'] = summary_name

!zip -r $SUMMARY_NAME $class_data_dir $class_validation_dir $validation_data_dir $OUTPUT_DIR $INSTANCE_DIR $instance_validation_dir

from google.colab import files
files.download(summary_name)

# Saves images every 5 epochs, en epoch er 5 steg, så hvert 5*5 steg lagres noe


# !accelerate launch train_dreambooth.py \
#   --pretrained_model_name_or_path=$MODEL_NAME  \
#   --instance_data_dir=$INSTANCE_DIR \
#   --output_dir=$OUTPUT_DIR \
#   --instance_prompt="a photo of sks dog" \
#   --resolution=512 \
#   --train_batch_size=1 \
#   --gradient_accumulation_steps=1 \
#   --learning_rate=5e-6 \
#   --lr_scheduler="constant" \
#   --lr_warmup_steps=0 \
#   --max_train_steps=400


2023-11-20 22:36:38.765808: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-20 22:36:38.765861: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-20 22:36:38.765893: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-20 22:36:39.944497: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
11/20/2023 22:36:41 - INFO - __main__ - Distributed environment: NO
Num processes: 1
Process index: 0
Local process index: 0
Device: cuda

Mixed precision type: no

You are using a model of type clip_text_model to instantiate a model of t

In [ ]:
!rm -r $OUTPUT_DIR
!rm -r $class_validation_dir
!rm -r $validation_data_dir
!rm -r $instance_validation_dir